<a href="https://colab.research.google.com/github/nishanth0233/INFO-5502/blob/main/Final_exam_Question_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### INFO 5502 Final Exam Question 3


<span style='background:yellow'>**Question 3 (100 pt, each sub-question is 10 pt).**</span> 

Part A: Given Inetd dataset, an intrusion detection dataset that contains 2 columns. The first column represents a process ID number (PID) and the second column represents a system call. The goal is to extract the entire system call sequence by its PID, then tokenize and parse each sequence into 25-grams. After this, we’ll split the data into training and testing sets. Next, we ensure data quality in the training dataset by removing duplicate sequences within each class and overlapped sequences between both classes. To ensure a fair evaluation, we also clean the testing set by repeating the previous step. You will implement a Decision Tree model to train and test on the training set and testing set that we just clean. 


Part B: As the data type is sequential, we want to know how similar the sequences from both classes are and visualize how the similarity scores vary in different sequence lengths. Compute a cosine similarity score of 100 random pairs from both classes and get a median cosine similarity. Repeat this step in different sequence lengths (5, 10, 15, and 20). Plot a line chart to visualize how the median cosine similarities vary through sequence lengths of 5 to 25. 

You can download the datasets from github: https://github.com/unt-iialab/info5502-spring2022/tree/main/finalexam/q4


Step by step instructions and questions:

(1) Attached are the Normal and Intrusion data folders. Read in the data and store them in two separate dataframes. Combine the system calls (the second column) from the same PID (the first column) into a sequence or a string. 

(2) Parse an entire sequence into smaller sequences of size 25. (Hint: Use word_tokenize and ngrams from nltk library to parse).

(3) Add labels to the data, and partition the labeled data into training data and testing data with a ratio of 70/30.

(4) As the dataset has a lot of duplicate sequences, make sure that the testing data only contain unique sequences and that these sequences do not exist in the training data. Condense training data and testing data into two sets. Take the difference between the testing set and training set to remove the overlapped sequences from the testing data. 

(5) The sequences from each class can be repetitive, and some of these sequences may have different labels. Therefore, we need to remove the duplicate sequences within each class and the overlapped sequences from the Intrusion class. Start out by separating the Normal class and Intrusion class in the training data. Use the same concept in Question 4 to remove duplicate sequences and overlapped sequences from the training data.

(6) Repeat Question 5 to clean the testing data.

(7) Bootstrap the training data to get the same number of sequences from each class. Train and evaluate a Decision Tree model using the training data and testing data that we just clean. How is the performance?

(8) Randomly sample 100 pairs of sequences from both classes and calculate the median cosine similarity.

(9) Refer back to Question 2, parse the data into a length of 5, 10, 15, and 20. Clean the data from each class using Question 5. For each sequence length, calculate the median cosine similarity of 100 random pairs of sequences from both classes. 

(10) Plot a line chart to visualize how the median cosine similarities vary through different sequence lengths from 5 to 25.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import numpy as np
import math

In [ ]:
normal=pd.read_csv('https://raw.githubusercontent.com/unt-iialab/info5502-spring2022/main/finalexam/q4/inetd-normal.int',sep=' ',header=None)
print(str("NORMAL DATASET"))
normal.columns = ['PID', 'SystemCalls']
print(normal.head().to_string())
intrusion=pd.read_csv('https://raw.githubusercontent.com/unt-iialab/info5502-spring2022/main/finalexam/q4/inetd-intrusion.int',sep=' ',header=None)
intrusion.columns = ['PID', 'SystemCalls']
print(str("INTRUSION DATASET"))
print(intrusion.head().to_string())

NORMAL DATASET
   PID  SystemCalls
0  167           90
1  167          125
2  167          125
3  167          106
4  167            5
INTRUSION DATASET
   PID  SystemCalls
0  167           90
1  167          125
2  167          125
3  167          106
4  167            5


In [ ]:
# Size of datasets using null values
print(normal.any().isna())
print('Normal Dataset has no null values\n')
print(intrusion.any().isna())
print('Intrusion dataset has no null values')

PID            False
SystemCalls    False
dtype: bool
Normal Dataset has no null values

PID            False
SystemCalls    False
dtype: bool
Intrusion dataset has no null values


In [ ]:
print(intrusion.shape)
print(normal.shape)

(8371, 2)
(541, 2)


In [ ]:
normal['PID'].value_counts()

2669    266
168     241
167      34
Name: PID, dtype: int64

In [ ]:
intr_dictionary = {}
norm_dictionary = {}

# combining system calls
for pid in list(set(intrusion['PID'].values)):
    intr_dictionary[pid] = " ".join(str(value) for value in list(intrusion[intrusion['PID'] == pid]['SystemCalls'].values))

for pid in list(set(normal['PID'].values)):
    norm_dictionary[pid] = " ".join(str(value) for value in list(normal[normal['PID'] == pid]['SystemCalls'].values))

intr_dictionary[167]

'90 125 125 106 5 90 6 5 3 90 90 90 90 6 125 91 125 125 125 136 49 24 47 50 45 45 5 106 106 106 106 106 2 1'

In [ ]:
import re
from nltk.util import ngrams
intr_list = []
norm_list = []

# generating ngrams with sequence length of 25

for key in intr_dictionary.keys():
    s = intr_dictionary[key]
    tokenz = [token for token in s.split(" ") if token != ""]
    
    for ele in list(ngrams(tokenz, 25)):
        intr_list.append(ele)
        
for key in norm_dictionary.keys():
    s = norm_dictionary[key]
    tokenz = [token for token in s.split(" ") if token != ""]
    
    for ele in list(ngrams(tokenz, 25)):
        norm_list.append(ele) 

print(len(intr_list))
print(len(norm_list))

7627
469


In [ ]:
intr_data = pd.DataFrame(intr_list)
norm_data = pd.DataFrame(norm_list)

# adding labels to intrusion and normal data sequences
intr_data['class'] = 1
norm_data['class'] = 0

print(intr_data.shape)
print(norm_data.shape)

(7627, 26)
(469, 26)


In [ ]:
# combining intrusion and normal data frames 

combined_data = pd.DataFrame()
combined_data = combined_data.append(intr_data)
combined_data = combined_data.append(norm_data)

print(combined_data.shape)

(8096, 26)


In [ ]:
# dropping duplicates...
combined_data = combined_data.drop_duplicates(subset = combined_data.columns[:-1], keep = 'last')
combined_data.shape

(333, 26)

In [ ]:
combined_data['class'].value_counts()
combined_data.columns[:-1]

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24],
      dtype='object')

In [ ]:
# building a decision tree classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

a = combined_data.drop("class", axis = 1)
b = combined_data['class']

a_train, a_valid, b_train, b_valid = train_test_split(a, b, test_size = 0.2, random_state = 0)

dataclassifier = DecisionTreeClassifier(max_depth = 10)
dataclassifier.fit(a_train, b_train)
b_pred = dataclassifier.predict(a_valid)

print(classification_report(b_valid, b_pred))

              precision    recall  f1-score   support

           0       0.79      0.86      0.83        22
           1       0.93      0.89      0.91        45

    accuracy                           0.88        67
   macro avg       0.86      0.88      0.87        67
weighted avg       0.88      0.88      0.88        67



# Similarity scores for NGram range of 5 to 25

In [ ]:
import pandas as pd
import numpy as np
import math
import re
from nltk.util import ngrams

cosine_similar_ngram = {}

for ngram in range(5, 26, 5):

    intrusion = pd.read_csv("https://raw.githubusercontent.com/unt-iialab/info5502-spring2022/main/finalexam/q4/inetd-intrusion.int", sep = " ", header = None)
    normal = pd.read_csv("https://raw.githubusercontent.com/unt-iialab/info5502-spring2022/main/finalexam/q4/inetd-normal.int", sep = " ", header = None)

    intrusion.columns = ['PID', 'SystemCalls']
    normal.columns = ['PID', 'SystemCalls']

    intr_dictionary = {}
    norm_dictionary = {}

    for pid in list(set(intrusion['PID'].values)):
        intr_dictionary[pid] = " ".join(str(value) for value in list(intrusion[intrusion['PID'] == pid]['SystemCalls'].values))

    for pid in list(set(normal['PID'].values)):
        norm_dictionary[pid] = " ".join(str(value) for value in list(normal[normal['PID'] == pid]['SystemCalls'].values))

    intr_list = []
    norm_list = []
    for key in intr_dictionary.keys():
        s = intr_dictionary[key]
        tokenz = [token for token in s.split(" ") if token != ""]

        for ele in list(ngrams(tokens, ngram)):
            intr_list.append(ele)

    for key in norm_dictionary.keys():
        s = norm_dictionary[key]
        tokenz = [token for token in s.split(" ") if token != ""]

        for ele in list(ngrams(tokenz, ngram)):
            norm_list.append(ele)

    intrusion_data = pd.DataFrame(intr_list)
    normal_data = pd.DataFrame(norm_list)

    intrusion_data['class'] = 1
    normal_data['class'] = 0

    combined_data = pd.DataFrame()

    combined_data = combined_data.append(intrusion_data)
    combined_data = combined_data.append(normal_data)

    print(combined_data.shape)
    print()


    combined_data = combined_data.drop_duplicates(subset = combined_df.columns[:-1], keep = 'last')
    print(combined_df.shape)
    print()

    from sklearn.metrics.pairwise import cosine_similarity
    import statistics

    similarities_list = []
    for i in range(0, 100):
        list1 = list(combined_df[combined_df['class'] == 0].sample(n = 1).iloc[0, :].values)
        list2 = list(combined_df[combined_df['class'] == 1].sample(n = 1).iloc[0, :].values)
        try:
            similarities_list.append(cosine_similarity([list1], [list2]))
        except Exception as e:
            print(e)
            print(list1, list2)

    print(statistics.median(similarities_list)[0][0])
    print()
    cosine_similar_ngram[ngram] = statistics.median(similarities_list)[0][0]
    